In [ ]:
import os

import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv("train.csv", index=False)

In [ ]:
def save_pred(y, sub_id):
    S.No,LABELS
    original_index = test_df.index
    output_df = pd.DataFrame(y, index=original_index, columns=["LABELS"], dtype=int)
    output_df.to_csv(f"./submission/submission{sub_id}.csv")